In [ ]:
# === MASTER HARMONIC TESTS (φ, triads, primes, sphere) ===
# One-cell, no installs. Produces plots + CSV + JSON.

import os, math, json, csv, time, cmath
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import default_rng

# ------------------------ Setup & Paths ------------------------
rng = default_rng(137)
plt.rcParams["figure.dpi"] = 130

BASE = "master_harmonic_tests"
PLOTS = f"{BASE}/plots"; ARTS = f"{BASE}/artifacts"
for d in [BASE, PLOTS, ARTS]: os.makedirs(d, exist_ok=True)

# Windows & caps
WINDOWS = [(10**6, 10**7), (10**7, 5*10**7)]
CAP_PRIMES = 250_000     # cap per window to keep runtime reasonable
NNULL_B = 100            # spherical harmonic null reps
phi = (1 + 5**0.5) / 2
NEAR_PHI = [1.60, 1.61, 1.62, 1.63]
BASES = [phi] + NEAR_PHI

# Families
FAMILIES = [("twin", 2), ("cousin", 4), ("sexy", 6)]

# Constants
alpha = 1/137.035999084
gear_g = (3**6)/(10**3)  # 0.729

# ------------------------ Prime utils ------------------------
def primes_in_window(lo, hi, cap=None):
    from sympy import primerange
    arr = np.fromiter(primerange(int(lo), int(hi)), dtype=np.int64)
    if cap and arr.size > cap:
        idx = rng.choice(arr.size, size=cap, replace=False)
        arr = arr[idx]
    return np.sort(arr)

def family_pairs(primes, d):
    pset = set(primes.tolist())
    out = [(p, p+d) for p in primes if (p+d) in pset]
    return np.array(out, dtype=np.int64)

# ------------------------ Stats utils ------------------------
def emd_hist_01(x, bins=180):
    x = (np.asarray(x, dtype=float) % 1.0)
    h,e = np.histogram(x, bins=bins, range=(0,1), density=True)
    c = (e[:-1]+e[1:])/2
    C = np.cumsum(h)
    if C[-1] == 0: return 0.0
    C /= C[-1]
    return float(np.trapezoid(np.abs(C - c), c))

def kuiper_p(x, m=1200):
    x = np.sort((np.asarray(x, dtype=float) % 1.0))
    n = len(x)
    if n == 0: return 0.0, 1.0
    grid = np.arange(1, n+1)/n
    Dp = np.max(grid - x)
    Dm = np.max(x - (np.arange(0, n)/n))
    V = Dp + Dm
    u  = np.sort(rng.random((m, n)), axis=1)
    Dp0 = np.max(grid - u, axis=1)
    u2 = np.sort(rng.random((m, n)), axis=1)
    Dm0 = np.max(u2 - (np.arange(0, n)/n), axis=1)
    V0 = Dp0 + Dm0
    p = (np.sum(V0 >= V) + 1)/(m + 1)
    return float(V), float(p)

# ------------------------ Phases / lenses ------------------------
def phi_phase(vals, base=phi, lens="log"):
    v = np.asarray(vals, dtype=float)
    v = v[np.isfinite(v) & (v > 0)]
    if v.size == 0: return np.array([])
    if lens == "id": y = v
    elif lens == "inv": y = 1.0/np.maximum(v, 1e-12)
    elif lens == "log": y = np.log(np.maximum(v, 1+1e-12))
    elif lens == "invlog": y = 1.0/np.log(np.maximum(v, 1+1e-12))
    else: y = np.log(np.maximum(v, 1+1e-12))
    return (np.log(v)/np.log(base)) % 1.0 if lens == "logphi" else (y*1.0) % 1.0

# ------------------------ Spherical harmonics ------------------------
# Try scipy.special.sph_harm if present; else fallback formula via lpmv if available.
try:
    from scipy.special import sph_harm as _scipy_sph_harm, lpmv
except Exception:
    _scipy_sph_harm, lpmv = None, None

def sph_harm_safe(m, l, theta, phi_colat):
    if _scipy_sph_harm is not None:
        return _scipy_sph_harm(m, l, theta, phi_colat)
    if lpmv is None:
        # very minimal fallback (not expected on Colab): approximate via numeric Y_l^m on grid
        # Use simple cosine series for m=0; degrade gracefully
        if m != 0:
            return np.zeros_like(theta, dtype=complex)
        P_l = np.polynomial.legendre.Legendre.basis(l)(np.cos(phi_colat))
        N = np.sqrt((2*l+1)/(4*np.pi))
        return (N*P_l).astype(complex)
    # Closed form via associated Legendre & phase:
    from math import factorial
    ct = np.cos(phi_colat)
    P_lm = lpmv(abs(m), l, ct)
    if m < 0:
        coef = ((-1)**m) * (math.factorial(l - m)/math.factorial(l + m))
        P_lm = coef * P_lm
    N = np.sqrt((2*l+1)/(4*np.pi) * math.factorial(l-abs(m))/math.factorial(l+abs(m)))
    return N * P_lm * np.exp(1j * m * theta)

def harmonic_energy(theta, phi_colat, Lmax=8):
    res = []
    for ell in range(1, Lmax+1):
        accum = 0.0
        for m in range(-ell, ell+1):
            Y = sph_harm_safe(m, ell, theta, phi_colat)
            accum += np.abs(np.mean(Y))**2
        res.append(float(accum.real))
    return np.array(res, dtype=float)

# ------------------------ FFT (windowed) ------------------------
def windowed_fft_power(x, seg=2048, hop=1024):
    x = np.asarray(x, dtype=float)
    if x.size < seg: return None
    W = []
    for i in range(0, x.size - seg + 1, hop):
        segx = x[i:i+seg]
        win = np.hanning(seg)
        X = np.fft.rfft(win*segx)
        W.append(np.abs(X)**2)
    if not W: return None
    S = np.mean(np.stack(W, axis=0), axis=0)  # average power
    f = np.fft.rfftfreq(seg, d=1.0)
    return f, S

def band_power(f, S, f0, bw=0.01):
    if f is None: return np.nan
    m = (f >= (f0-bw)) & (f <= (f0+bw))
    if not np.any(m): return np.nan
    return float(np.mean(S[m]))

# ------------------------ KMeans (k<=3) minimal ------------------------
def kmeans_1d(values, k, iters=30):
    v = np.asarray(values, dtype=float).reshape(-1,1)
    if v.shape[0] < k: k = max(1, v.shape[0])
    # init with quantiles
    qs = np.linspace(0.0, 1.0, k+2)[1:-1]
    centers = np.quantile(v, qs, axis=0)
    for _ in range(iters):
        d = np.abs(v - centers.T)
        lbl = np.argmin(d, axis=1)
        newc = np.array([v[lbl==j].mean() if np.any(lbl==j) else centers[j] for j in range(k)]).reshape(-1,1)
        if np.allclose(newc, centers): break
        centers = newc
    centers = np.sort(centers.ravel())
    return centers

# ------------------------ Fibit (Fibonacci word -> decimal) ------------------------
def fib_word_digits(N=4096):
    a,b = "0","01"
    s = a
    while len(s) < N:
        a,b = b, a+b
        s = a
    return s[:N]

def fibit_value(N=4096):
    s = fib_word_digits(N)
    pw = 10.0**(-np.arange(1, len(s)+1, dtype=float))
    bits = np.fromiter((1 if ch=='1' else 0 for ch in s), dtype=float, count=len(s))
    return float(np.dot(bits, pw))

# ------------------------ Master containers ------------------------
report = {
    "windows": WINDOWS,
    "spherical": [],
    "mod3_phi": [],
    "triadic_fft": [],
    "three_gap": [],
    "gear": []
}

# ================================================================
# (1) Spherical harmonics: ℓ=5 pentagon + ℓ=3/6 sidebands
# ================================================================
for (lo, hi) in WINDOWS:
    primes = primes_in_window(lo, hi, cap=CAP_PRIMES)
    # Embed with φ, φ^2
    th = (2*np.pi * ((np.log(primes)*phi) % 1.0)).astype(float)
    phcol = np.arccos(2*((np.log(primes)*(phi**2)) % 1.0)-1.0).astype(float)
    E = harmonic_energy(th, phcol, Lmax=8)

    # Nulls
    sh_E, un_E = [], []
    for _ in range(NNULL_B):
        idx = rng.permutation(primes.size)
        sh_E.append(harmonic_energy(th, phcol[idx], Lmax=8))
    for _ in range(NNULL_B):
        u = rng.random(primes.size); v = rng.random(primes.size)
        th_u = 2*np.pi*u
        ph_u = np.arccos(2*v-1.0)
        un_E.append(harmonic_energy(th_u, ph_u, Lmax=8))
    sh_E = np.array(sh_E); un_E = np.array(un_E)
    nulls = np.vstack([sh_E, un_E])
    mu = nulls.mean(axis=0); sd = nulls.std(axis=0) + 1e-12
    Z = (E - mu)/sd

    # Plots
    ells = np.arange(1,9)
    plt.figure()
    plt.plot(ells, Z, marker='o')
    plt.axhline(4, ls='--', lw=0.8)
    plt.title(f"Spherical harmonics z-scores — [{lo},{hi}) (φ-embed)")
    plt.xlabel("ℓ"); plt.ylabel("z-score")
    plt.xticks(ells)
    plt.tight_layout()
    plt.savefig(f"{PLOTS}/sph_z_{lo}_{hi}.png"); plt.close()

    pent_ok = (Z[4] >= 4.0)  # ℓ=5 index 4
    tri_ok  = ((Z[2] >= 2.0) or (Z[5] >= 2.0))  # ℓ=3 or ℓ=6
    report["spherical"].append({
        "window":[lo,hi],
        "z_l": {int(ell): float(z) for ell,z in zip(ells,Z)},
        "pent_ok": bool(pent_ok),
        "tri_ok": bool(tri_ok)
    })

# ================================================================
# (2) Residue-filtered φ signal (mod 3), EMD & Kuiper vs near-φ
# ================================================================
mod3_rows = []
for (lo, hi) in WINDOWS:
    primes = primes_in_window(lo, hi, cap=CAP_PRIMES)
    primes = primes[primes != 3]
    for rclass in [1, -1]:
        cls = primes[primes % 3 == (rclass % 3)]
        # Series: (A) class bases themselves; (B) gaps between them
        bases = cls
        gaps = np.diff(bases) if bases.size>1 else np.array([])
        for series_name, seq in [("bases", bases), ("gaps", gaps)]:
            # φ & near-φ
            stats = []
            for b in BASES:
                if series_name == "gaps":
                    phases = (np.log(np.maximum(seq, 2)) / np.log(b)) % 1.0
                else:
                    phases = (np.log(np.maximum(seq, 3)) / np.log(b)) % 1.0
                emd = emd_hist_01(phases, bins=160)
                _, kp = kuiper_p(phases, m=1200)
                stats.append((b, emd, kp))
            # ranks / margins
            emds = np.array([s[1] for s in stats])
            max_b = stats[int(np.argmax(emds))][0]
            min_b = stats[int(np.argmin(emds))][0]
            # φ metrics
            phi_idx = [i for i,s in enumerate(stats) if abs(s[0]-phi)<1e-12][0]
            phi_emd = stats[phi_idx][1]; phi_kp = stats[phi_idx][2]
            # margins vs neighbors (near-φ only)
            near_emds = np.array([s[1] for s in stats if s[0] in NEAR_PHI])
            max_near = float(np.max(near_emds)) if near_emds.size else np.nan
            min_near = float(np.min(near_emds)) if near_emds.size else np.nan
            margin_as_max = float(phi_emd - max_near) if near_emds.size else np.nan
            margin_as_min = float(min_near - phi_emd) if near_emds.size else np.nan

            mod3_rows.append([lo,hi,rclass,series_name,phi_emd,phi_kp,
                              float(max_b), float(min_b),
                              margin_as_max, margin_as_min])

# Save & small plot
with open(f"{ARTS}/mod3_phi_summary.csv","w") as f:
    w = csv.writer(f);
    w.writerow(["lo","hi","mod3_class","series","phi_emd","phi_kuiper_p",
                "best_base_by_emd","worst_base_by_emd","phi_margin_as_max","phi_margin_as_min"])
    w.writerows(mod3_rows)

report["mod3_phi"] = mod3_rows

# ================================================================
# (3) Base-3 spectral peaks: fibit & prime-family event trains
# ================================================================
def event_train_from_bases(bases, lo, hi, cap_len=200000):
    # map bases to an index grid of length L; compress by stride
    if bases.size == 0: return np.zeros(1)
    span = hi - lo
    L = min(cap_len, span)
    x = np.zeros(L, dtype=float)
    idx = ((bases - lo) * (L/span)).astype(int)
    idx = idx[(idx>=0) & (idx<L)]
    x[idx] = 1.0
    return x

triad_rows = []
for (lo, hi) in WINDOWS:
    primes = primes_in_window(lo, hi, cap=CAP_PRIMES)
    # Families event trains
    fam_signals = {}
    for name,d in FAMILIES:
        pairs = family_pairs(primes, d)
        bases = pairs[:,0] if pairs.size else np.array([], dtype=np.int64)
        sig = event_train_from_bases(bases, lo, hi, cap_len=200000)
        fam_signals[name] = sig

    # Fibit signals (two variants): raw bits 0/1 and balanced (-1,0,+1) by mapping 1->+1, 0->0
    Nf = 1<<13  # 8192 digits to keep fast
    fw = fib_word_digits(Nf)
    fib_raw = np.array([1 if ch=='1' else 0 for ch in fw], dtype=float)
    fib_bal = fib_raw.copy()  # keep 0/1 for simplicity here

    # Compute windowed FFT power bands around 1/3, 1/6, 1/9
    def triad_powers(sig):
        fS = windowed_fft_power(sig, seg=2048, hop=1024)
        if fS is None: return {"f1_3":np.nan,"f1_6":np.nan,"f1_9":np.nan}
        f,S = fS
        return {
            "f1_3": band_power(f,S,1/3,0.012),
            "f1_6": band_power(f,S,1/6,0.012),
            "f1_9": band_power(f,S,1/9,0.01),
        }

    # Collect
    entries = []
    entries.append(("fibit_raw", triad_powers(fib_raw)))
    entries.append(("fibit_bal", triad_powers(fib_bal)))
    for name,sig in fam_signals.items():
        entries.append((name, triad_powers(sig)))

    # Save & bar plot per window
    plt.figure()
    labels = []
    b13=[]; b16=[]; b19=[]
    for name,p in entries:
        labels.append(name); b13.append(p["f1_3"]); b16.append(p["f1_6"]); b19.append(p["f1_9"])
        triad_rows.append([lo,hi,name,p["f1_3"],p["f1_6"],p["f1_9"]])
    x = np.arange(len(labels))
    w = 0.25
    plt.bar(x- w, b13, width=w, label="~1/3")
    plt.bar(x+0.0, b16, width=w, label="~1/6")
    plt.bar(x+ w, b19, width=w, label="~1/9")
    plt.xticks(x, labels, rotation=20)
    plt.ylabel("Avg band power")
    plt.title(f"Triadic bands (FFT, windowed) — [{lo},{hi})")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{PLOTS}/triadic_fft_{lo}_{hi}.png"); plt.close()

with open(f"{ARTS}/triadic_fft_summary.csv","w") as f:
    w = csv.writer(f); w.writerow(["lo","hi","signal","band_1_3","band_1_6","band_1_9"])
    w.writerows(triad_rows)

report["triadic_fft"] = triad_rows

# ================================================================
# (4) Three-gap fingerprints under φ rotations (vs null)
# ================================================================
threegap_rows = []
for (lo, hi) in WINDOWS:
    primes = primes_in_window(lo, hi, cap=CAP_PRIMES)
    # Use twin-bases as the sequence (a family with structure)
    pairs = family_pairs(primes, 2)
    bases = pairs[:,0] if pairs.size else np.array([], dtype=np.int64)
    sequences = [("twin_bases", bases)]

    # Also include residue class +1 mod 3 bases
    cls1 = primes[(primes % 3)==1]
    sequences.append(("mod3_plus1_bases", cls1))

    # Null: Cramér on [lo,hi) then select same count
    xs = np.arange(lo, hi, dtype=np.int64)
    probs = 1.0/np.log(np.maximum(xs, 3))
    hits = rng.random(xs.size) < probs
    pp = xs[hits]
    # twin-like "bases" from null (pairs by +2)
    pset = set(pp.tolist())
    null_twins = np.array([p for p in pp if (p+2) in pset], dtype=np.int64)
    sequences.append(("null_twin_bases", null_twins))

    for name,seq in sequences:
        if seq.size < 20:
            threegap_rows.append([lo,hi,name,0,0,0,np.nan,np.nan]); continue
        # φ-phase, sort, circle gaps
        theta = (np.log(np.maximum(seq,3))/np.log(phi)) % 1.0
        ths = np.sort(theta)
        gaps = np.diff(np.r_[ths, ths[0]+1.0])
        # k-means with k<=3
        k = 3 if gaps.size>=3 else min(2, gaps.size)
        centers = kmeans_1d(gaps, k=k, iters=30)
        centers = np.sort(centers)
        k_used = centers.size
        ratio = float(centers[-1]/centers[0]) if (k_used>=2 and centers[0]>0) else np.nan

        # Store; a goldenish hint if ratio ≈ φ or simple function (we just record ratio)
        threegap_rows.append([lo,hi,name, gaps.size, k_used] + list(centers[:3]) + [ratio])

        # Plot histogram of gaps + centers
        plt.figure()
        plt.hist(gaps, bins=100, density=True, alpha=0.8)
        for c in centers: plt.axvline(c, color='k', lw=0.8, ls='--')
        plt.title(f"Circle gaps under φ-phase — {name} [{lo},{hi})")
        plt.xlabel("gap length"); plt.ylabel("density")
        plt.tight_layout()
        plt.savefig(f"{PLOTS}/threegap_{name}_{lo}_{hi}.png"); plt.close()

with open(f"{ARTS}/threegap_summary.csv","w") as f:
    w = csv.writer(f)
    w.writerow(["lo","hi","sequence","n_gaps","k_used","center1","center2","center3","ratio_max_over_min"])
    w.writerows(threegap_rows)

report["three_gap"] = threegap_rows

# ================================================================
# (5) Gear specificity scan around g = 3^6/10^3
# ================================================================
gear_rows = []
for N in [1024, 2048, 4096]:
    fv = fibit_value(N)
    # Scan λ in structured ternary->decimal set and fine real grid
    cand = []
    for k in range(4,9):
        for m in range(2,5):
            cand.append((3**k)/(10**m))
    fine = np.linspace(gear_g-0.01, gear_g+0.01, 81)
    lambdas = sorted(set([float(x) for x in cand] + [float(x) for x in fine] + [gear_g]))
    for lam in lambdas:
        y = lam * fv
        F1 = abs(y*1e3 - alpha)  # | (λ·fibit)*10^3 - α |
        gear_rows.append([N, lam, y, F1])

# Save & small plot of F1 vs λ near g (for N=4096)
with open(f"{ARTS}/gear_scan.csv","w") as f:
    w = csv.writer(f); w.writerow(["fibit_digits","lambda","lambda_times_fibit","F1_alignment_error"])
    w.writerows(gear_rows)

sel = [r for r in gear_rows if r[0]==4096 and abs(r[1]-gear_g)<=0.01]
sel = sorted(sel, key=lambda r:r[1])
plt.figure()
plt.plot([r[1] for r in sel], [r[3] for r in sel], marker='o')
plt.axvline(gear_g, ls='--', lw=0.8)
plt.title("Gear scan near g=3^6/10^3 — F1=| (λ·fibit)·10^3 − α | (N=4096)")
plt.xlabel("λ"); plt.ylabel("alignment error F1")
plt.tight_layout()
plt.savefig(f"{PLOTS}/gear_scan_near_g.png"); plt.close()

report["gear"] = {
    "note":"See gear_scan.csv; F1 minimized candidates near g appear as local minima if present.",
    "g": gear_g
}

# ------------------------ Write master report ------------------------
with open(f"{ARTS}/master_report.json","w") as f:
    json.dump(report, f, indent=2)

print(json.dumps({
    "done": True,
    "outputs": {
        "plots_dir": PLOTS,
        "artifacts_dir": ARTS,
        "files": [
            "sph_z_[windows].png",
            "mod3_phi_summary.csv",
            "triadic_fft_[windows].png",
            "threegap_summary.csv",
            "gear_scan.csv",
            "master_report.json"
        ]
    }
}, indent=2))


/tmp/ipython-input-845505777.py:93: DeprecationWarning: `scipy.special.sph_harm` is deprecated as of SciPy 1.15.0 and will be removed in SciPy 1.17.0. Please use `scipy.special.sph_harm_y` instead.
  return _scipy_sph_harm(m, l, theta, phi_colat)


{
  "done": true,
  "outputs": {
    "plots_dir": "master_harmonic_tests/plots",
    "artifacts_dir": "master_harmonic_tests/artifacts",
    "files": [
      "sph_z_[windows].png",
      "mod3_phi_summary.csv",
      "triadic_fft_[windows].png",
      "threegap_summary.csv",
      "gear_scan.csv",
      "master_report.json"
    ]
  }
}
